In [81]:
from os import path
from collections import deque

from ga4stpg.graph import UGraph
from ga4stpg.graph.disjointsets import DisjointSets
from ga4stpg.graph.priorityqueue import PriorityQueue
from ga4stpg.graph.reader import read_problem
from ga4stpg.graph.util import has_cycle, how_many_components
from ga4stpg.util import STEIN_B

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST
from ga4stpg.tree.mutate import ReplaceByRandomEdge
from ga4stpg.tree.pxcrossover import PXTree

In [82]:
INDEX_SELECT = 4

datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(datasets, STEIN_B[INDEX_SELECT][0])

print(filename)

STPG = read_problem(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinb5.txt


In [83]:
from ga4stpg.edgeset import UEdge, EdgeSet

class Segment:

    def __init__(self):
        self.edges = EdgeSet()
        self.cost = 0
        self.portals = set()

    def __len__(self):
        return len(self.edges)

    def __str__(self):
        return f'Segment <{len(self.edges)}>'

    def __iter__(self):
        return iter(self.edges)

    @property
    def bounds(self):
        return frozenset(self.portals)
    
    def add(self, v, u):
        self.edges.add(UEdge(v, u))
    

In [128]:
from random import choice

def f_weight(v, u):
    if STPG.graph.has_edge(v, u):
        return STPG.graph.weight(v, u)
    else:
        return 0

def find_segments(graph, common_nodes):
    visited = set()
    stack_outter = [choice(tuple(common_nodes))]
    result = list()

    def search(start, neighbor):
        segment = Segment()
        segment.portals.add(start)
        segment.add(start, neighbor)
        # segment.cost += f_weight(start, neighbor)
        
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                counter = 0
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add(u, w)
                        # segment.cost += f_weight(u, w)
                        counter += 1
                if counter == 0:
                    segment.portals.add(u)                    
            else:
                stack_outter.append(u)
                segment.portals.add(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                seg = search(s, v)
                result.append(seg)

    return result

In [129]:
common_nodes = {1, 17, 5, 13, 9}

edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9),
    (7, 23),
]

red = UGraph()

for v, u in edges:
    red.add_edge(v, u)

In [130]:
segments = find_segments(red, common_nodes)

In [131]:
for seg in segments:
    print(seg, seg.portals)
    for edge in seg:
        print(edge, end="  ")
    print("\n\n")

Segment <2> {17, 1}
Edge <(1, 15)>  Edge <(15, 17)>  


Segment <2> {1, 5}
Edge <(1, 3)>  Edge <(3, 5)>  


Segment <2> {13, 5}
Edge <(11, 13)>  Edge <(5, 11)>  


Segment <3> {9, 5, 23}
Edge <(5, 7)>  Edge <(7, 23)>  Edge <(7, 9)>  




## Crossing two random tree generated by Prim Random Steneir Tree

In [88]:
generator = GenerateBasedPrimRST(STPG)

red  = generator()
blue = generator()

In [89]:
red.edges == blue.edges

False

In [90]:
child     = UGraph()
red_only  = UGraph()
blue_only = UGraph()

In [91]:
for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v, u)
    else:
        red_only.add_edge(v, u)

In [92]:
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

In [93]:
how_many_components(child)

12

In [94]:
common_nodes = set(red_only.vertices) & set(blue_only.vertices)
terminals = STPG.terminals.copy()

common_nodes & terminals

{3, 5, 23, 37}

In [95]:
len(common_nodes)

21

In [96]:
red_segments  = find_segments(red_only, common_nodes)
blue_segments = find_segments(blue_only, common_nodes)

len(red_segments)

19

In [97]:
len(blue_segments)

16

In [98]:
queue = PriorityQueue()

for seg in red_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

for seg in blue_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

len(queue)

35

In [99]:
dsets = DisjointSets()

for v in child.vertices:
    dsets.make_set(v)

for v in common_nodes:
    if v not in dsets:
        dsets.make_set(v)

for v, u in child.gen_undirect_edges():
    dsets.union(v, u)

In [100]:
dsets.get_disjoint_sets()

defaultdict(set,
            {31: {16, 23, 29, 31, 42, 43, 44, 45},
             48: {26, 48},
             18: {6, 18},
             47: {46, 47},
             35: {1, 7, 12, 15, 35},
             14: {14, 25},
             28: {2, 28},
             40: {10, 13, 40},
             49: {4, 21, 49},
             24: {24, 36},
             32: {30, 32},
             20: {3, 20, 39},
             5: {5},
             8: {8},
             9: {9},
             11: {11},
             17: {17},
             27: {27},
             37: {37},
             38: {38}})

In [101]:
def check_portals(portals, disjoint):
    '''Verifica se os vértices portais de um segmento
    se conectam à mesma partição de vértices comuns. 

    Faz essa verificação em tempo O(n)
    '''
    f_check = set()

    for p in portals:
        if p not in disjoint:
            return False
        k = disjoint.find(p)
        if k in f_check:
            return False
        f_check.add(k)

    return True

In [102]:
while queue:
    seg = queue.pop()
    test = set()

    # if len(seg.portals) == 1:
    #     continue

    if check_portals(seg.portals, dsets):
        # print('adding segment', seg)
        for edge in seg:
            v, u = edge
            child.add_edge(v, u)
        portals = list(seg.portals)
        for p1, p2 in zip(portals[0::2], portals[1::2]):
            dsets.union(p1, p2)
            # print(p1, p2, dsets.find(p1), dsets.find(p2))
    else:
        print("####### >>> rejecting: ", seg)

####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <2>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <7>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <2>


In [103]:
for seg in red_segments:
    parent = dict()
    for p in seg.portals:
        if p in dsets:
            parent[p] = dsets.find(p)

    print(parent)

{17: 47, 46: 47}
{18: 47, 46: 47}
{38: 47, 46: 47}
{18: 47, 5: 47}
{17: 47, 27: 47}
{17: 47, 12: 47}
{8: 47, 45: 47}
{37: 47}
{21: 47, 37: 47}
{3: 47, 21: 47}
{21: 47, 6: 47}
{6: 47, 31: 47}
{9: 47, 36: 47}
{9: 47, 25: 47}
{25: 47, 43: 47}
{25: 47, 30: 47}
{32: 47, 11: 47}
{10: 47, 23: 47}
{2: 28}


In [104]:
for seg in blue_segments:
    parent = dict()
    for p in seg.portals:
        if p in dsets:
            parent[p] = dsets.find(p)
    print(parent)

{36: 47, 46: 47}
{32: 47, 36: 47}
{8: 47, 36: 47}
{8: 47, 37: 47}
{37: 47}
{32: 47, 35: 47, 5: 47, 6: 47, 9: 47, 27: 47}
{6: 47}
{25: 47, 5: 47}
{48: 47, 27: 47}
{45: 47, 38: 47}
{11: 47, 30: 47}
{23: 47}
{17: 47, 21: 47}
{18: 47, 10: 47}
{3: 47, 14: 47}
{2: 28}


In [105]:
has_cycle(child)

True

In [106]:
how_many_components(child)

2

In [107]:
seg

In [108]:
for seg in blue_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{36, 46}
Edge <(36, 46)>  
{32, 36}
Edge <(32, 36)>  
{8, 36}
Edge <(8, 36)>  
{8, 37}
Edge <(8, 37)>  
{19, 37}
Edge <(19, 37)>  
{32, 35, 5, 6, 9, 27}
Edge <(42, 49)>  Edge <(5, 49)>  Edge <(6, 42)>  Edge <(9, 42)>  Edge <(27, 42)>  Edge <(32, 42)>  Edge <(35, 49)>  
{50, 6}
Edge <(6, 50)>  
{25, 5}
Edge <(5, 25)>  
{48, 27}
Edge <(27, 48)>  
{45, 38}
Edge <(38, 45)>  
{11, 30}
Edge <(11, 30)>  
{33, 23}
Edge <(23, 33)>  
{17, 21}
Edge <(17, 21)>  
{18, 10}
Edge <(10, 18)>  
{3, 14}
Edge <(3, 14)>  
{41, 2}
Edge <(4, 41)>  Edge <(2, 4)>  
